### Solution to the February 2023 Jane Street puzzle: Twenty Four Seven (Four-in-One)


### https://www.janestreet.com/puzzles/lesses-more-index/

In [ ]:
# Solve the February 2023 Jane Street puzzle: Twenty Four Seven (Four-in-One)
# using the PuLP linear programming package.
# I couldn't find a nice way to formulate the constraints relating to the value of the first/last number in a row or column
# as a linear problem (although it is definitely possible), 
# therefore several cell values have to be calculated manually and entered to warm-start the solver;

from pulp import *

# The labels of each row and column of the 12x12 grid, ranging from 1 to 12
rows = cols = range(1, 13)
# The possible non-zero values for each cell, ranging from 1 to 7.
vals = range(1, 8)
# Create the model
model = LpProblem("24/7_Problem")
# Create a 12x12 set of integer variables that can range from 0 to 7 i.e the 12x12 grid
x = LpVariable.dicts("x", (rows, cols), cat="Integer", lowBound=0,  upBound=7)
# Create a 12x12x7 set of binary variables
y =  LpVariable.dicts("y", (vals, rows, cols), cat="Binary")


# Set constraints for each 7x7 subgrid
for i in range(2):
    for j in range(2):
        
        # Constraint: On each subgrid, for each distinct value 0<z<8, ensure that it occurs with frequency = z i.e seven sevens etc
        for v in vals:
            model += lpSum(y[v][r][c] for r in rows[i*5:i*5+7] for c in cols[j*5:j*5+7]) == v
            
        # Constraint: Couple the integer and binary variables
        for r in rows[i*5:i*5+7]:
            for c in cols[j*5:j*5+7]:
                
                model += lpSum(v * y[v][r][c] for v in vals) == x[r][c]
                           
        # Constraint: On each subgrid, ensure that for each row, sum(row) = 20
        for r in rows[i*5:i*5+7]:
            model += lpSum(x[r][c] for c in cols[j*5:j*5+7]) == 20 
                           
        # Constraint: On each subgrid, ensure that for each column, sum(column) = 20
        for c in cols[j*5:j*5+7]:
            model += lpSum(x[r][c] for r in rows[i*5:i*5+7]) == 20
                           
        # Constraint: On each subgrid, ensure that for each row, the frequency of non-zero values = 4
        for r in rows[i*5:i*5+7]:
            model += lpSum(y[v][r][c] for v in vals[0:7] for c in cols[j*5:j*5+7]) == 4
                           
        # Constraint: On each subgrid, ensure that for each column, the frequency of non-zero values = 4
        for c in cols[j*5:j*5+7]:
            model += lpSum(y[v][r][c] for v in vals for r in rows[i*5:i*5+7]) == 4

# All possible 2x2 subgrids                          
subgrid_2x2ss = [
    [(i + k , j + l) for k in range(2) for l in range(2)]
    for i in range(1, 12)
    for j in range(1, 12)
]

# Constraint: Each 2x2 subgrid must contain at minimum, one zero valued cell (or 3 non-zero valued cells)                         
for g in subgrid_2x2ss:
        model += lpSum(lpSum(y[v][r][c] for v in vals) for (r, c) in g) <= 3
                           
# Constraint: The numbers placed outside the grid, constraining the sum of the entire row
model += lpSum(x[4][c] for c in cols) == 33
model += lpSum(x[5][c] for c in cols) == 29
model += lpSum(x[7][c] for c in cols) == 40
model += lpSum(x[8][c] for c in cols) == 28
model += lpSum(x[11][c] for c in cols) == 36
    
# Constraint: The numbers placed outside the grid, constraining the sum of the entire column
model += lpSum(x[r][2] for r in rows) == 36
model += lpSum(x[r][3] for r in rows) == 30
model += lpSum(x[r][4] for r in rows) == 34
model += lpSum(x[r][5] for r in rows) == 27
model += lpSum(x[r][7] for r in rows) == 40
model += lpSum(x[r][8] for r in rows) == 27

# (weak) Constraint: The numbers placed outside the grid, constraining the first/last non-zero element of the row
model += lpSum(y[5][1][c] for c in cols[0:3]) >= 1
model += lpSum(y[4][1][c] for c in cols[11:12]) >= 1
model += lpSum(y[7][2][c] for c in cols[0:3]) >= 1
model += lpSum(y[7][3][c] for c in cols[0:4]) >= 1
model += lpSum(y[1][4][c] for c in cols[9:12]) >= 1
model += lpSum(y[2][6][c] for c in cols[0:2]) >= 1
model += lpSum(y[7][12][c] for c in cols[8:12]) >= 1

# (weak) Constraint: The numbers placed outside the grid, constraining the first/last non-zero element of the column
model += lpSum(y[6][r][1] for r in rows[0:4]) >= 1
model += lpSum(y[6][r][1] for r in rows[8:12]) >= 1
model += lpSum(y[4][r][4] for r in rows[9:12]) >= 1
model += lpSum(y[3][r][6] for r in rows[0:2]) >= 1
model += lpSum(y[7][r][11] for r in rows[0:1]) >= 1
model += lpSum(y[5][r][12] for r in rows[8:12]) >= 1


# Constraint: The initial numbers placed inside the grid   
subgrid_a = [(1,2,4),(3,3,6),(4,4,2),(2,5,5),(6,6,3)]
subgrid_a = subgrid_a + [(0,1,1),(0,2,1),(0,3,1),(6,4,1),(4,7,3),(2,6,1),(0,5,2),(0,5,3),(0,5,4),(5,7,1)]

subgrid_b = [(6,2,10),(5,2,11),(6,3,11),(7,4,9),(7,5,12),(3,6,9),(7,6,10)]
subgrid_b = subgrid_b + [(7,1,11),(4,1,12),(0,1,10),(0,2,12),(0,3,10)]

subgrid_c = [(7,9,4),(5,9,6),(5,10,2),(7,10,6),(6,11,2),(7,11,3),(6,12,5)]
subgrid_c = subgrid_c + []

subgrid_d = []
subgrid_d = subgrid_d + [(0,12,12),(0,11,12)]

initial_data = subgrid_a + subgrid_b + subgrid_c + subgrid_d

for (v, r, c) in initial_data:
    model += x[r][c] == v
                           
model.writeLP("model.lp")
model.solve(PULP_CBC_CMD(threads = 16,msg=0,maxSeconds = 15))

# Print out 12x12 grid
for r in rows:
    for c in cols:
        print(int(value(x[r][c])), end="     ")
    print("\n")

0     5     0     6     0     3     6     0     0     0     7     4     

0     7     7     1     0     0     5     0     4     6     5     0     

0     0     0     7     5     3     5     0     6     0     6     0     

6     4     3     0     0     7     0     5     7     1     0     0     

7     0     0     0     2     7     4     2     0     0     0     7     

2     0     6     6     6     0     0     6     3     7     0     4     

5     4     4     0     7     0     0     7     0     6     2     5     

7     0     0     0     1     5     7     1     0     0     7     0     

0     5     3     7     0     5     0     0     5     0     4     6     

6     5     0     0     0     7     2     0     6     0     0     5     

0     6     7     3     0     0     4     6     6     4     0     0     

0     0     0     4     6     3     7     0     0     3     7     0     

